In [3]:
import sympy as sp

def norm(x, y, z):
    return sp.sqrt(x**2 + y**2 + z**2)

def dot(ax, ay, az, bx, by, bz):
    return ax*bx + ay*by + az*bz

def imc_friction_sym(x0, x1, x2, y0, y1, y2, a0, a1, a2, b0, b1, b2,
        xp0, xp1, xp2, yp0, yp1, yp2, ap0, ap1, ap2, bp0, bp1, bp2,
        xf0, xf1, xf2, yf0, yf1, yf2, af0, af1, af2, bf0, bf1, bf2,
        mu, dt, vel_tol):

    K2 = 15 / vel_tol

    # Norms
    f1s_n = norm(xf0, xf1, xf2)
    f1e_n = norm(yf0, yf1, yf2)
    f2s_n = norm(af0, af1, af2)
    f2e_n = norm(bf0, bf1, bf2)
    f1_n = norm(xf0 + yf0, xf1 + yf1, xf2 + yf2)
    f2_n = norm(af0 + bf0, af1 + bf1, af2 + bf2)

    # Contact point weights
    t1 = f1s_n / f1_n
    t2 = 1 - t1
    u1 = f2s_n / f2_n
    u2 = 1 - u1

    # Contact normal
    contact_nx = (xf0 + yf0) / f1_n
    contact_ny = (xf1 + yf1) / f1_n
    contact_nz = (xf2 + yf2) / f1_n

    # Velocities
    v1sx = (x0 - xp0) / dt
    v1sy = (x1 - xp1) / dt
    v1sz = (x2 - xp2) / dt
    v1ex = (y0 - yp0) / dt
    v1ey = (y1 - yp1) / dt
    v1ez = (y2 - yp2) / dt
    v2sx = (a0 - ap0) / dt
    v2sy = (a1 - ap1) / dt
    v2sz = (a2 - ap2) / dt
    v2ex = (b0 - bp0) / dt
    v2ey = (b1 - bp1) / dt
    v2ez = (b2 - bp2) / dt

    v1x = t1 * v1sx + t2 * v1ex
    v1y = t1 * v1sy + t2 * v1ey
    v1z = t1 * v1sz + t2 * v1ez
    v2x = u1 * v2sx + u2 * v2ex
    v2y = u1 * v2sy + u2 * v2ey
    v2z = u1 * v2sz + u2 * v2ez

    # Relative velocity and tangential projection
    vrx = v1x - v2x
    vry = v1y - v2y
    vrz = v1z - v2z
    dot_vn = dot(vrx, vry, vrz, contact_nx, contact_ny, contact_nz)
    tx = vrx - dot_vn * contact_nx
    ty = vry - dot_vn * contact_ny
    tz = vrz - dot_vn * contact_nz
    tv_norm = norm(tx, ty, tz)

    # Unit tangent vector
    tx_u = tx / tv_norm
    ty_u = ty / tv_norm
    tz_u = tz / tv_norm

    # Gamma modulation for sticking
    gamma = (2 / (1 + sp.exp(-K2 * tv_norm))) - 1

    # --- STICKING FRICTION ---
    ffr1x_s = gamma * mu * tx_u
    ffr1y_s = gamma * mu * ty_u
    ffr1z_s = gamma * mu * tz_u
    ffr2x_s = -ffr1x_s
    ffr2y_s = -ffr1y_s
    ffr2z_s = -ffr1z_s

    ffr_vec1 = [
        ffr1x_s * f1s_n, ffr1y_s * f1s_n, ffr1z_s * f1s_n,
        ffr1x_s * f1e_n, ffr1y_s * f1e_n, ffr1z_s * f1e_n,
        ffr2x_s * f2s_n, ffr2y_s * f2s_n, ffr2z_s * f2s_n,
        ffr2x_s * f2e_n, ffr2y_s * f2e_n, ffr2z_s * f2e_n
    ]

    # --- SLIDING FRICTION ---
    ffr1x_sl = mu * tx_u
    ffr1y_sl = mu * ty_u
    ffr1z_sl = mu * tz_u
    ffr2x_sl = -ffr1x_sl
    ffr2y_sl = -ffr1y_sl
    ffr2z_sl = -ffr1z_sl

    ffr_vec2 = [
        ffr1x_sl * f1s_n, ffr1y_sl * f1s_n, ffr1z_sl * f1s_n,
        ffr1x_sl * f1e_n, ffr1y_sl * f1e_n, ffr1z_sl * f1e_n,
        ffr2x_sl * f2s_n, ffr2y_sl * f2s_n, ffr2z_sl * f2s_n,
        ffr2x_sl * f2e_n, ffr2y_sl * f2e_n, ffr2z_sl * f2e_n
    ]

    return ffr_vec1, ffr_vec2


In [4]:
from sympy import lambdify

def generate_friction_jacobian_funcs():
    import sympy as sp
    vars = sp.symbols(
        'x0 x1 x2 y0 y1 y2 a0 a1 a2 b0 b1 b2 '
        'xp0 xp1 xp2 yp0 yp1 yp2 ap0 ap1 ap2 bp0 bp1 bp2 '
        'xf0 xf1 xf2 yf0 yf1 yf2 af0 af1 af2 bf0 bf1 bf2 mu dt vel_tol', real=True
    )
    ffr_stick, ffr_slide = imc_friction_sym(*vars)

    pos_vars = vars[:12]      # current node positions (x, y, a, b)
    force_vars = vars[24:36]  # current contact forces (f1s, f1e, f2s, f2e)
    all_inputs = vars

    Jn_stick = sp.Matrix(ffr_stick).jacobian(pos_vars)   # ∂friction / ∂x
    Jf_stick = sp.Matrix(ffr_stick).jacobian(force_vars) # ∂friction / ∂f

    Jn_slide = sp.Matrix(ffr_slide).jacobian(pos_vars)   # ∂friction / ∂x
    Jf_slide = sp.Matrix(ffr_slide).jacobian(force_vars) # ∂friction / ∂f


    dfr_dx_func = lambdify(all_inputs, Jn_stick, modules='numpy', cse=True, docstring_limit=0)
    dfr_df_func = lambdify(all_inputs, Jf_stick, modules='numpy', cse=True, docstring_limit=0)

    dfr_dx_func2 = lambdify(all_inputs, Jn_slide, modules='numpy', cse=True, docstring_limit=0)
    dfr_df_func2 = lambdify(all_inputs, Jf_slide, modules='numpy', cse=True, docstring_limit=0)

    return dfr_dx_func, dfr_df_func, dfr_dx_func2, dfr_df_func2

dfr_dx_func, dfr_df_func, dfr_dx_func2, dfr_df_func2 = generate_friction_jacobian_funcs()

In [5]:
import numpy as np

def make_test_input():
    # Current positions of 4 nodes (forming 2 rods)
    x1s = [0.0, 0.0, 0.0]
    x1e = [1.0, 0.0, 0.0]
    x2s = [0.5, 0.1, 0.0]
    x2e = [1.5, 0.1, 0.0]

    # Previous positions (moved slightly to create velocity)
    x1s0 = [0.0, 0.0, 0.0]
    x1e0 = [1.0, 0.0, 0.0]
    x2s0 = [0.5, 0.09, 0.0]  # small tangential motion
    x2e0 = [1.5, 0.09, 0.0]

    # Contact forces
    f1s = [0.0, 0.0, 1.0]
    f1e = [0.0, 0.0, 1.0]
    f2s = [0.0, 0.0, -1.0]
    f2e = [0.0, 0.0, -1.0]

    mu = 0.5         # friction coefficient
    dt = 0.01        # timestep
    vel_tol = 1e-3   # velocity smoothing threshold

    # Combine all inputs into one list
    return np.array(
        x1s + x1e + x2s + x2e +
        x1s0 + x1e0 + x2s0 + x2e0 +
        f1s + f1e + f2s + f2e +
        [mu, dt, vel_tol],
        dtype=float
    )

In [10]:
def compute_friction_jacobian(inputs, contact_jacobian, 
                               dfr_dx_func, dfr_df_func, 
                               friction_type="Sticking", constraint_type="EdgeToEdge"):
    """
    Compute full Jacobian of friction force w.r.t. current node positions using chain rule.

    Parameters:
    - inputs: (39,) array of all symbolic inputs
    - contact_jacobian: (12, 12) np.ndarray, ∂f_contact / ∂x
    - dfr_dx_func: function, returns ∂f_friction / ∂x from sympy
    - dfr_df_func: function, returns ∂f_friction / ∂f from sympy
    - friction_type: "Sticking" or "Sliding"
    - constraint_type: "EdgeToEdge", "PointToEdge", or "PointToPoint"

    Returns:
    - Jfr: (12, 12) np.ndarray, full Jacobian of friction force w.r.t. node positions
    """
    assert inputs.shape[0] == 39, "Input must be a (39,) vector"

    # Get Jacobians (evaluate lambdified functions)
    Jx = np.array(dfr_dx_func(*inputs))  # ∂f_friction / ∂x
    Jf = np.array(dfr_df_func(*inputs))  # ∂f_friction / ∂f_contact

    # Apply constraint-type masking to Jf (match MATLAB logic)
    Jf = Jf.copy()
    if constraint_type == "PointToPoint":
        Jf[:, 3:6] = 0.0   # zero f1e
        Jf[:, 9:12] = 0.0  # zero f2e
    elif constraint_type == "PointToEdge":
        Jf[:, 3:6] = 0.0   # zero f1e only

    # Chain rule
    Jfr = Jf @ contact_jacobian + Jx

    # Optional sanity check
    if np.isnan(Jfr).any():
        raise ValueError("IMC friction Jacobian contains NaNs")

    return Jfr


In [12]:
inputs = make_test_input()

# Evaluate Jacobians for STICKING mode
Jc = np.eye(12)  # For testing: identity (∂f_contact/∂x)

Jfr = compute_friction_jacobian(
    inputs,
    contact_jacobian=Jc,
    dfr_dx_func=dfr_dx_func,
    dfr_df_func=dfr_df_func,
    friction_type="Sticking",
    constraint_type="EdgeToEdge"
)

print("Full Jacobian shape:", Jfr.shape)
Jfr

Full Jacobian shape: (12, 12)


array([[ 2.50000000e+01,  0.00000000e+00,  0.00000000e+00,
         2.50000000e+01,  0.00000000e+00,  0.00000000e+00,
        -2.50000000e+01,  0.00000000e+00,  0.00000000e+00,
        -2.50000000e+01,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  3.55271368e-15, -5.00000000e-01,
         0.00000000e+00,  3.55271368e-15,  0.00000000e+00,
         0.00000000e+00, -3.55271368e-15,  0.00000000e+00,
         0.00000000e+00, -3.55271368e-15,  0.00000000e+00],
       [ 0.00000000e+00,  2.50000000e-01,  0.00000000e+00,
         0.00000000e+00,  2.50000000e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.50000000e+01,  0.00000000e+00,  0.00000000e+00,
         2.50000000e+01,  0.00000000e+00,  0.00000000e+00,
        -2.50000000e+01,  0.00000000e+00,  0.00000000e+00,
        -2.50000000e+01,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  3.55271368e-15,  0.0000000